In [ ]:
import torch
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import tinycudann as tcnn

from nemo.nemo import Nemo
from nemo.util import grid_2d

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%load_ext autoreload
%autoreload 2

In [ ]:
torch.tensor([0,1]) + torch.rand(10,2)

In [ ]:
torch.ones(10,1).roll(1,0)

In [ ]:
# Get data to fit
nemo = Nemo()
nemo.load_weights('../models/kt22_encs.pth', '../models/kt22_heightnet.pth')

xy = grid_2d(N=64, bounds=(-1, 1, -1, 1))
xy = xy.to(device)
z = nemo.get_heights(xy)

In [ ]:
# New Nemo
nemo = Nemo()

# Encodings

https://github.com/NVlabs/tiny-cuda-nn/blob/master/DOCUMENTATION.md

### Hashgrid

* `n_levels` - number of levels in the hashgrid. Increasing leads to better fit but noisier gradients.
* `n_features_per_level` - number of features per level. Can be 1, 2, 4 or 8.


In [ ]:
nemo.encoding = tcnn.Encoding(
    n_input_dims=2,
    encoding_config={
        "otype": "HashGrid",
        "n_levels": 8,
        "n_features_per_level": 8,
        "log2_hashmap_size": 19,
        "base_resolution": 16,
        "per_level_scale": 1.2599210739135742,
    },
)

Fit to data

In [ ]:
tot_out_dims_2d = nemo.encoding.n_output_dims
print(tot_out_dims_2d)

nemo.height_net = tcnn.Network(
    n_input_dims=tot_out_dims_2d,
    n_output_dims=1,
    network_config={
        "otype": "CutlassMLP",
        "activation": "Softplus",
        "output_activation": "None",
        "n_neurons": 256,
        "n_hidden_layers": 1,
    },
)

In [ ]:
nemo.height_net

In [ ]:
nemo.encoding.to(device)
nemo.height_net.to(device)
nemo.fit(xy.detach(), z.detach())

In [ ]:
nemo.plot(N=512)

In [ ]:
N = 512
bounds = (-1., 1., -1., 1)
positions = grid_2d(N, bounds)
positions.requires_grad = True
z_pred, grad = nemo.get_heights_with_grad(positions)

x_grad = grad[:,0].reshape(N, N).detach().cpu().numpy()
y_grad = grad[:,1].reshape(N, N).detach().cpu().numpy()

#x_grad = np.clip(x_grad, -2.0, 2.0)

fig = go.Figure()
fig.add_trace(go.Heatmap(z=x_grad))
fig.update_layout(width=600, height=600, scene_aspectmode='data')
fig.show()

In [ ]:
# Finite difference gradient
h = 5e-2
positions_dx = positions.clone()
positions_dx[:,0] += h
z_dx = nemo.get_heights(positions_dx)
grad_x_fd = (z_dx - z_pred) / h
grad_x_fd = grad_x_fd.clip(-1.0, 1.0)

fig = go.Figure()
fig.add_trace(go.Heatmap(z=grad_x_fd.reshape(N, N).detach().cpu().numpy()))
fig.update_layout(width=600, height=600, scene_aspectmode='data')
fig.show()

# MLP 

In [ ]:
nemo = Nemo()

### ReLU

In [ ]:
tot_out_dims_2d = nemo.encoding.n_output_dims
print(tot_out_dims_2d)

nemo.height_net = tcnn.Network(
    n_input_dims=tot_out_dims_2d,
    n_output_dims=1,
    network_config={
        "otype": "CutlassMLP",
        "activation": "ReLU",
        "output_activation": "None",
        "n_neurons": 256,
        "n_hidden_layers": 1,
    },
)

### Softplus

In [ ]:
tot_out_dims_2d = nemo.encoding.n_output_dims
print(tot_out_dims_2d)

nemo.height_net = tcnn.Network(
    n_input_dims=tot_out_dims_2d,
    n_output_dims=1,
    network_config={
        "otype": "CutlassMLP",
        "activation": "ReLU",
        "output_activation": "None",
        "n_neurons": 256,
        "n_hidden_layers": 1,
    },
)